In [1]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import re
from tqdm import tqdm
from os import listdir
import pdb
import dill
from datetime import datetime
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json
import time


In [2]:
from gensim import corpora, models, similarities, matutils, utils
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import scipy.interpolate as interpolate
from sklearn.decomposition import PCA
from sklearn.cluster import k_means
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import metrics
from nltk import FreqDist
import pandas as pd
import matplotlib.cm as cm
import numpy as np
import time
import nltk
import pdb
import re
import os

C:\Users\br41n5\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tokenizer = RegexpTokenizer("\w+")
stop_words=set(stopwords.words('english'))
p_stemmer = PorterStemmer()

In [20]:
data_path=os.getcwd()+'\\website\\flask_files\\model_db\\'
subs_dir=os.getcwd()+'\\subs_storage\\'
import pickle
def save_obj(obj, name):
    with open(data_path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, protocol=2)
        
def load_obj(name):
    with open(data_path + name + '.pkd', 'rb') as fp:
        return pickle.load(fp)

In [49]:
corp_list

['tt0002381',
 'tt0004189',
 'tt0004282',
 'tt0004284',
 'tt0004288',
 'tt0004395',
 'tt0005044',
 'tt0006177',
 'tt0006548',
 'tt0006684',
 'tt0007162',
 'tt0007264',
 'tt0007507',
 'tt0007832',
 'tt0011508',
 'tt0011841',
 'tt0012255',
 'tt0012349',
 'tt0013486',
 'tt0014358',
 'tt0015324',
 'tt0015841',
 'tt0015863',
 'tt0015864',
 'tt0016220',
 'tt0017765',
 'tt0017925',
 'tt0018051',
 'tt0018455',
 'tt0018773',
 'tt0018839',
 'tt0019071',
 'tt0019109',
 'tt0019421',
 'tt0020629',
 'tt0020768',
 'tt0020960',
 'tt0021025',
 'tt0021079',
 'tt0021735',
 'tt0021749',
 'tt0021814',
 'tt0021884',
 'tt0022208',
 'tt0022251',
 'tt0022286',
 'tt0022626',
 'tt0022698',
 'tt0022827',
 'tt0022835',
 'tt0022883',
 'tt0022913',
 'tt0022958',
 'tt0023042',
 'tt0023194',
 'tt0023245',
 'tt0023293',
 'tt0023427',
 'tt0023470',
 'tt0023551',
 'tt0023825',
 'tt0023858',
 'tt0023876',
 'tt0023926',
 'tt0023940',
 'tt0023969',
 'tt0023993',
 'tt0024006',
 'tt0024166',
 'tt0024184',
 'tt0024216',
 'tt00

In [46]:
if []:
    print(caca)

In [11]:
with open(data_path+'movie_DB_1_7_2018_B.pkl', 'rb') as fp:
    movie_database = dill.load(fp)

In [11]:
def get_title(imdb_id):
    try:
        ind=[item['imdb_id'] for item in movie_database].index(imdb_id)
        return movie_database[ind]['title'] 
    except:
        return 'Not Found.'

def read_srt(imdb_id):
    if os.path.isfile(subs_dir+imdb_id + '.srt'):
        with open(subs_dir+imdb_id+'.srt','rb') as fp:
            test=fp.read().decode('utf-8')
        return test
    else:
        pass

def fparse1(string):
    if string:
        #string=re.sub(r'<i>','',string)    
        #string=re.sub(r'</i>','',string)
        string=re.sub('<[^>]*.','',string) #kills anything inside < CACA >
        string=re.sub(r'\ufeff','',string)
        #string=re.sub('^[^(1)]*','',string) #removes stuff before the 1
        string=re.sub('^[^\a]*?1\r\n00:','1\r\n00:',string) #removes stuff before the 1 betterly
        return string
    else:
        pass

def srt_conv(subs):
    if subs:
        starttimes=[]
        endtimes=[]
        words=[]
        indexes=[]
        #"-->" in line
        subs=io.StringIO(subs)
        prevLineWasWords =False
        prevLineWasTime =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
                prevLineWasTime =True
            elif line.strip().isdigit() and not prevLineWasTime:
                indexes.append(int(line))
                prevLineWasTime=False
            else:
            #elif len(line) > 1:
                prevLineWasTime = False
                if prevLineWasWords:
                        words[len(words)-1] = words[len(words)-1] + " " + line
                else:
                        words.append(line)
                prevLineWasWords = True
                continue
            prevLineWasWords = False  
        
        #trim because sometimes a file will end without a last tine.
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]
        
        return starttimes, endtimes, words, indexes
    else:
        pass
    
def watch_output(clean_srt):
    subs=io.StringIO(clean_srt)
    print('=== is number, ~~~ is text, --- is time index. \n')
    prevLineWasTime=False
    for line in subs:
        if "-->" in line:
            prevLineWasTime=True
            print('----'+line)
        elif line.strip().isdigit() and not prevLineWasTime:
            prevLineWasTime=False
            print('====='+line)
        else:
        #elif len(line)>1:
            prevLineWasTime=False
            print('~~~~'+line)
    return

def s_time(string):
    try:
        a=datetime.strptime(string, '%H:%M:%S')
        return a
    except:
        pass
    

def make_df(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        data_sub=pd.DataFrame(columns=index)


        s1 = pd.Series([s_time(item) for item in a])
        s2 = pd.Series([s_time(item) for item in b])
        s3 = pd.Series(c)
        data_sub.loc[:,(imdb_id,'StartTime')] =s1.values
        data_sub.loc[:,(imdb_id,'EndTime')] = s2.values
        data_sub.loc[:,(imdb_id,'Line')] = s3.values

        return data_sub
    else:
        pass
    
    
def make_tup(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        s1 = [s_time(item) for item in a]
        s2 = [s_time(item) for item in b]
        s3 = c

        return (s1,s2,s3)
    else:
        pass    

def append_pd(df1,df):
    tes=pd.concat([df1,df], axis=1)
    tes=tes.fillna('')
    return tes

# Parsing subtitles - text files

In [82]:
subdb_cur=[item.split('.')[0] for item in listdir(subs_dir) if 'tt' in item]

In [12]:
with open(data_path+'subdb_cur.pkd',"rb") as fp:
    subdb_cur=dill.load(fp)

In [9]:
def parse(string):
    #give me a string and ill return you a tokenized, stop_worded, stemmed, list of words greater than 2
    string=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '',string)
    aa=[p_stemmer.stem(w) for w in tokenizer.tokenize(string) if not w in stop_words]
    return aa

t=time.time()
raw={}
for i in range(len(subdb_cur)):
    raw[subdb_cur[i]]=parse(' '.join(make_tup(subdb_cur[i])[2]))
    if i%100==0:
        print('No. {} - adding {} - elapsed {} sec.'.format(i,subdb_cur[i],time.time()-t))
t1=time.time()
print('elapsed time {} seconds'.format(t1-t))

No. 0 - adding tt0002381 - elapsed 0.006036281585693359 sec.
No. 100 - adding tt0026685 - elapsed 7.6426615715026855 sec.
No. 200 - adding tt0032811 - elapsed 21.18127965927124 sec.
No. 300 - adding tt0037522 - elapsed 35.09215950965881 sec.
No. 400 - adding tt0041487 - elapsed 50.20632576942444 sec.
No. 500 - adding tt0045468 - elapsed 64.1695876121521 sec.
No. 600 - adding tt0048728 - elapsed 77.4451596736908 sec.
No. 700 - adding tt0052549 - elapsed 91.41245770454407 sec.
No. 800 - adding tt0056085 - elapsed 105.69560432434082 sec.
No. 900 - adding tt0059243 - elapsed 119.01004314422607 sec.
No. 1000 - adding tt0062512 - elapsed 131.26176524162292 sec.
No. 1100 - adding tt0065938 - elapsed 144.1050226688385 sec.
No. 1200 - adding tt0068646 - elapsed 155.47674560546875 sec.
No. 1300 - adding tt0071650 - elapsed 167.09701490402222 sec.
No. 1400 - adding tt0075232 - elapsed 179.0064618587494 sec.
No. 1500 - adding tt0078111 - elapsed 190.49855589866638 sec.
No. 1600 - adding tt0080755 

In [15]:
with open(data_path+'raw_subs.pkl','rb') as fp:
    raw=pickle.load(fp)

In [16]:
sys.getsizeof(raw)

589928

In [17]:
print('--==word statistics==--')
pd.Series([len(item) for item in list(raw.values())]).describe()

--==word statistics==--


count    11659.000000
mean      4560.205850
std       1984.865001
min          0.000000
25%       3264.500000
50%       4400.000000
75%       5696.500000
max      31872.000000
dtype: float64

# NLP Model

In [19]:
texts=list(raw.values())
dictionary=corpora.Dictionary(texts)
print('Original dictionary size: {}'.format(len(dictionary.token2id)))
min_term_freq=1
max_term_fraction=0.3
dictionary.filter_extremes(no_below=min_term_freq,no_above=max_term_fraction,keep_n=200000)
print('Filtered extremes, dictionary size: {}'.format(len(dictionary.token2id)))
dictionary.save(data_path+'dictionary.dict')  #save dictionary

Original dictionary size: 248049
Filtered extremes, dictionary size: 200000


In [20]:
corpus=[dictionary.doc2bow(text) for text in texts]

In [21]:
t2=time.time()
tfidf_mod=models.TfidfModel(corpus)
corpus_tfidf=tfidf_mod[corpus]


lsi_mod=models.LsiModel(corpus_tfidf, 
                        id2word=dictionary,
                        num_topics=150,
                        onepass=False,
                        power_iters=30)
corpus_lsi=lsi_mod[corpus_tfidf]

print('Elapsed time: {}'.format(time.time()-t2))

Elapsed time: 682.5529391765594


In [45]:
if [1]:
    print('aa')

aa


In [22]:
index=similarities.MatrixSimilarity(corpus_lsi)

In [23]:
corpora.MmCorpus.serialize(data_path+'corpus_lsi.mm', corpus_lsi)  #save the corpus
lsi_mod.save(data_path+'lsi_model.lsi') #save the model
tfidf_mod.save(data_path+'tfidf_mod.lsi') #save the model
index.save(data_path+'similarity.index') #save the index

AttributeError: 'TransformedCorpus' object has no attribute 'size'

# Reco System

In [35]:
find_titles_available('matrix')

[('the matrix', 'tt0133093', 3964),
 ('the matrix reloaded', 'tt0234215', 4433),
 ('the matrix revolutions', 'tt0242653', 4473),
 ('the animatrix', 'tt0328832', 5026),
 ('the matrix revisited', 'tt0295432', 4813)]

In [36]:
corpus_lsi[3964]

[(0, 0.0066573399095585248),
 (1, 0.038348896079226542),
 (2, 0.021719523224612243),
 (3, -0.0039045038578385954),
 (4, 0.020512644871868178),
 (5, 0.010731144570563811),
 (6, 0.030046230360544535),
 (7, 0.0002410509844502992),
 (8, 0.0062770810094271521),
 (9, -0.0043326982444001731),
 (10, 0.011425911182261012),
 (11, 0.0010334087012338336),
 (12, 0.0090533130960472),
 (13, -0.0009881303781613779),
 (14, 0.020801179076596694),
 (15, 0.0075696920730312037),
 (16, 0.00059427415901062216),
 (17, -0.0013355158290884641),
 (18, 0.0054826758896429385),
 (19, -0.0056570152142246142),
 (20, -0.0075970530016745792),
 (21, -0.0099850284009173236),
 (22, -0.00084778403561684256),
 (23, 0.0023086042922036147),
 (24, -0.0014633995377078212),
 (25, 0.0061434736506518935),
 (26, -0.0093959104953662172),
 (27, 0.00050404486259384459),
 (28, 0.00073497259396838638),
 (29, -0.0034224543138713479),
 (30, 0.001190573442054365),
 (31, 0.0023582141066155653),
 (32, 0.0081737658909570004),
 (33, 0.00930841

In [68]:
def find_in_corpora(imdb_id):
    return subdb_cur.index(imdb_id)

def find_title_by_imdbid(imdbid):
    return [item['title'] for item in movie_database if item['imdb_id'] == imdbid][0]


def continuity_check(corpora_id):
    if corp_list[corpora_id] in [item['imdb_id'] for item in movie_database]:
        return True
    else:
        return False
    
def find_titles_available(query):
    #input any part of the title, lower or uppercase
    #reutrns a list of possible meanings (plus imdbid, location in corpora)
    #all of which
    #have available subtitles
    #if no results returns EMPTY LIST
    tts=[(item['title'].lower(),item['imdb_id']) for item in movie_database]
    #movies in the movie database
    subdb_cur=[item.split('.')[0] for item in listdir(subs_dir) if 'tt' in item]
    #movies in the SUB database
    return [(s[0],s[1],find_in_corpora(s[1])) for s in tts if query.lower() in s[0] and s[1] in subdb_cur]

def return_movie_info(corpora_id_list):
    tts=[(item['title'].lower(),item['imdb_id']) for item in movie_database]
    subdb_cur=[item.split('.')[0] for item in listdir(subs_dir) if 'tt' in item]
    if isinstance(corpora_id_list, int):
        corpora_id_list=[corpora_id_list]
        
    if isinstance(corpora_id_list, list):
        imdbids=[(find_title_by_imdbid(subdb_cur[item]),subdb_cur[item]) for item in corpora_id_list]
        return imdbids
    
def rec_me(imdb_id,num):
    movie=raw[imdb_id]
    vec_movie=lsi_mod[tfidf_mod[dictionary.doc2bow(movie)]]
    index.num_best=num
    sims=index[vec_movie]
    #print(sims)
    sims = [item for item in sims if continuity_check(item[0])]
    
    corp_list=list(raw.keys())
    return [(find_title_by_imdbid(corp_list[item[0]]),corp_list[item[0]]) for item in sims]


def search(string):
    results=find_titles_available(string)
    results.sort(key = lambda s: len(s[0]))
    for i in range(len(results)):
        print('{} - {}.'.format(i, results[i][0].title()))
    choice=int(input('Multiple titles. Which one?'))
    print('You chose {}. Finding recommendations.\n'.format(results[choice][0].title()))
    recomendations=rec_me(results[choice][1],10)
    for item in recomendations: print(item[0])
    results=[[get_path(item[1]),get_title(item[1]),get_year(item[1]),get_score(item[1])] for item in recomendations]
    return results

import pdb
def search2(string):
    results=find_titles_available(string)
    results.sort(key = lambda s: len(s[0]))
    for i in range(len(results)):
        print('{} - {}.'.format(i, results[i][0].title()))
    choice=int(input('Multiple titles. Which one?'))
    print('You chose {}. Finding recommendations.\n'.format(results[choice][0].title()))
    recomendations=rec_me2(results[choice][2],10)
    for item in recomendations: print(item[0])
    results=[[get_path(item[1]),get_title(item[1]),get_year(item[1]),get_score(item[1])] for item in recomendations]
    return results



def rec_me2(corpus_id,num):
    sims=index[corpus_lsi[corpus_id]]
    corp_list=list(raw.keys())
    sims = [item for item in sims if continuity_check(item[0])]
    #pdb.set_trace()
    return [(find_title_by_imdbid(corp_list[item[0]]),corp_list[item[0]]) for item in sims]



def get_path(imdb_id):
    image_path='https://image.tmdb.org/t/p/w300'
    return image_path+[item['poster_path'] for item in movie_database if item['imdb_id'] == imdb_id][0]
def get_year(imdb_id):
    return [item['release_date'] for item in movie_database if item['imdb_id'] == imdb_id][0]
def get_score(imdb_id):
    return [item['vote_average'] for item in movie_database if item['imdb_id'] == imdb_id][0]
def get_title(imdb_id):
    return [item['title'] for item in movie_database if item['imdb_id'] == imdb_id][0]


def make_link(data):
    link= "<img src='" + data[0]+ "' alt='missing' />"
    name= "<figcaption>" + data[1]
    year="<br>"+"Year "+ data[2]
    vote= "<br>"+"Score "+str(data[3])+"</figcaption>"
    return "<figure>"+link+name+year+vote+"</figure>"

In [69]:
results = search2('despicable me')

0 - Despicable Me.
1 - Despicable Me 2.
2 - Despicable Me 3.
Multiple titles. Which one?0
You chose Despicable Me. Finding recommendations.

Despicable Me
Megamind
Hands of Stone
Ghostbusters II
Dr. Horrible's Sing-Along Blog
DC's Legends of Tomorrow: Their Time Is Now
Escape Plan
The Founder
A Dirty Shame


In [55]:
print()

<figure><img src='https://image.tmdb.org/t/p/w300/hEpWvX6Bp79eLxY1kX5ZZJcme5U.jpg' alt='missing' /><figcaption>The Matrix<br>Year 1999-03-30<br>Score 8.0</figcaption></figure>
<figure><img src='https://image.tmdb.org/t/p/w300/ezIurBz2fdUc68d98Fp9dRf5ihv.jpg' alt='missing' /><figcaption>The Matrix Reloaded<br>Year 2003-05-15<br>Score 6.7</figcaption></figure>
<figure><img src='https://image.tmdb.org/t/p/w300/56Jw3jcT0Oi8dRI2htlCQyfRlV8.jpg' alt='missing' /><figcaption>Teleios<br>Year 2017-04-06<br>Score 5.1</figcaption></figure>
<figure><img src='https://image.tmdb.org/t/p/w300/ecz0Dz5IP9Ouk7Ta4WCOM7TxXua.jpg' alt='missing' /><figcaption>Lo and Behold: Reveries of the Connected World<br>Year 2016-03-04<br>Score 6.5</figcaption></figure>
<figure><img src='https://image.tmdb.org/t/p/w300/8cie5mojY6MlIrYMs9EtNSyterv.jpg' alt='missing' /><figcaption>THX 1138<br>Year 1971-03-11<br>Score 6.4</figcaption></figure>
<figure><img src='https://image.tmdb.org/t/p/w300/uRPlPIKJmqiH4TioXouU9G8dx48.jp

In [74]:
np.random.randint(100)

20

In [60]:
search2('despicable me')

0 - Despicable Me.
1 - Despicable Me 2.
2 - Despicable Me 3.
Multiple titles. Which one?0
You chose Despicable Me. Finding recommendations.



IndexError: list index out of range